# Building and Evaluating N-gram Language Models

**BSc Data Science — CDS 3400: Natural Language Processing**


**Contents:**
- Theory and motivation
- Dataset (Shakespeare Sonnets)
- Preprocessing
- Building unigram, bigram, trigram models
- Add-one smoothing (Laplace)
- Interpolation
- Text generation
- Perplexity evaluation

---

## Notebook instructions

This notebook is designed to be run top-to-bottom. It uses **NLTK + custom Python code** (no advanced deep-learning libraries). The notebook will:

1. Download the Shakespeare sonnets from Project Gutenberg.
2. Preprocess the text and split into train/test sets.
3. Build counts for unigram, bigram, and trigram models.
4. Implement add-one smoothing and an interpolation model.
5. Generate "Shakespeare-like" lines.
6. Calculate perplexity on a held-out test set.

 `shakespeare_sonnets.txt`.

---

In [ ]:
# Setup: imports and basic helpers
import re, os, random, math
from collections import defaultdict, Counter
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import ngrams
from collections import Counter
from wordcloud import WordCloud

import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

import matplotlib.pyplot as plt


# Ensure required NLTK packages are available (these lines will attempt to download if missing)
nltk_packages = ['punkt', 'averaged_perceptron_tagger', 'stopwords']
for pkg in nltk_packages:
    try:
        nltk.data.find(f'tokenizers/{pkg}')
    except LookupError:
        try:
            nltk.download(pkg)
        except:
            pass

print("Imports ready. NLTK packages attempted (downloaded if missing).")

Imports ready. NLTK packages attempted (downloaded if missing).


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

# 1) Load Shakespeare sonnets text
# Primary attempt: download from Project Gutenberg
url = "https://www.gutenberg.org/files/1041/1041-0.txt"
local_filename = "shakespeare_sonnets.txt"

def download_sonnets(url=url, local_filename=local_filename):
    try:
        import requests
        print("Attempting to download sonnets from Project Gutenberg...")
        r = requests.get(url, timeout=20)
        r.raise_for_status()
        text = r.text
        with open(local_filename, "w", encoding="utf-8") as f:
            f.write(text)
        print(f"Downloaded and saved to {local_filename}.")
        return local_filename
    except Exception as e:
        print("Automatic download failed:", e)
        if os.path.exists(local_filename):
            print(f"Found local file {local_filename}. Using it.")
            return local_filename
        else:
            print("No local copy found. Please download the sonnets from Project Gutenberg and save as 'shakespeare_sonnets.txt' in the working directory.")
            raise

try:
    data_path = download_sonnets()
except Exception as exc:
    # If running in an environment without internet, raise to show the user what to do.
    raise RuntimeError("Could not download or locate the sonnets file. Please place 'shakespeare_sonnets.txt' in the working directory.") from exc

# Read the file
with open(data_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Loaded text length:", len(raw_text))

Attempting to download sonnets from Project Gutenberg...
Downloaded and saved to shakespeare_sonnets.txt.
Loaded text length: 96312


In [ ]:
# 2) Preprocessing: isolate sonnets and clean text
# Project Gutenberg sonnets file contains copyright headers. We'll try to extract the sonnets section heuristically.

def extract_sonnets(text):
    # Heuristic: split by "THE SONNETS" or by "SHAKESPEARE" headings, otherwise just use whole text
    low = text.lower()
    if "sonnets" in low:
        # attempt to find the start of the sonnets section
        m = re.search(r"(?:sonnets)|(the sonnets)", low)
        start = m.start() if m else 0
        excerpt = text[start:]
    else:
        excerpt = text
    # Split into lines and keep those that look like sonnet lines (skip Gutenberg headers/footers)
    lines = [ln.strip() for ln in excerpt.splitlines() if ln.strip()]
    # Attempt to find lines that include Roman numerals (sonnet numbers) or "Sonnet" words, otherwise fallback
    # For safety, we'll use a simple filter to remove Gutenberg header/footer lines
    filtered = [ln for ln in lines if len(ln) > 1 and not ln.lower().startswith('project gutenberg') and 'gutenberg' not in ln.lower()]
    return "\\n".join(filtered)

sonnets_text = extract_sonnets(raw_text)

# Basic cleanup and sentence splitting
# We'll use a simple tokenizer: split into sentences via NLTK, then into word tokens.


sentences = sent_tokenize(sonnets_text)
print(f"Extracted {len(sentences)} sentences (raw heuristic).")

# Lowercase and remove non-word characters except apostrophes and spaces
def clean_text(s):
    s = s.lower()
    # keep letters, numbers, apostrophes and spaces
    s = re.sub(r"[^a-z0-9'\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

clean_sentences = [clean_text(s) for s in sentences if len(clean_text(s).split()) >= 1]

# Show a sample
for i, s in enumerate(clean_sentences[:6]):
    print(i+1, "-", s)

Extracted 76 sentences (raw heuristic).
1 - the sonnets nby william shakespeare nfrom fairest creatures we desire increase nthat thereby beauty s rose might never die nbut as the riper should by time decease nhis tender heir might bear his memory nbut thou contracted to thine own bright eyes nfeed st thy light s flame with self substantial fuel nmaking a famine where abundance lies nthyself thy foe to thy sweet self too cruel nthou that art now the world s fresh ornament nand only herald to the gaudy spring nwithin thine own bud buriest thy content nand tender churl mak st waste in niggarding npity the world or else this glutton be nto eat the world s due by the grave and thee nii nwhen forty winters shall besiege thy brow nand dig deep trenches in thy beauty s field nthy youth s proud livery so gazed on now nwill be a tatter d weed of small worth held nthen being asked where all thy beauty lies nwhere all the treasure of thy lusty days nto say within thine own deep sunken eyes nwere a

In [ ]:

# 3) Train-test split and build counts for unigrams, bigrams, trigrams
random.seed(42)
sentences_shuffled = clean_sentences.copy()
random.shuffle(sentences_shuffled)

# Use ~80% train, 20% test (by sentences)
split_idx = int(0.8 * len(sentences_shuffled))
train_sents = sentences_shuffled[:split_idx]
test_sents = sentences_shuffled[split_idx:]

print(f"Train sentences: {len(train_sents)}, Test sentences: {len(test_sents)}")

def tokenize_sentence(s):
    return s.split()  # already cleaned and lowercased

# Build counts
unigram_counts = Counter()
bigram_counts = Counter()
trigram_counts = Counter()
vocab = set()

for sent in train_sents:
    tokens = tokenize_sentence(sent)
    vocab.update(tokens)
    # add start tokens for context handling
    tokens_padded = ["<s>"] + tokens + ["</s>"]
    for i in range(len(tokens_padded)):
        if i >= 1:
            unigram_counts[tokens_padded[i]] += 1
        if i >= 2:
            bigram = (tokens_padded[i-1], tokens_padded[i])
            bigram_counts[bigram] += 1
        if i >= 3:
            trigram = (tokens_padded[i-2], tokens_padded[i-1], tokens_padded[i])
            trigram_counts[trigram] += 1

V = len(vocab)
print(f"Vocabulary size (train): {V}")
print("Top 10 unigrams:", unigram_counts.most_common(10))

Train sentences: 60, Test sentences: 16
Vocabulary size (train): 3034
Top 10 unigrams: [('my', 279), ('the', 273), ('to', 266), ('of', 265), ('i', 254), ('in', 240), ('thy', 218), ('and', 204), ('that', 199), ('nand', 192)]


In [ ]:

# 4) Probability estimators: MLE, Add-one smoothing, Interpolation
def mle_unigram(w):
    total = sum(unigram_counts.values())
    return unigram_counts[w] / total if total > 0 else 0.0

def mle_bigram(w_prev, w):
    denom = unigram_counts[w_prev] if unigram_counts[w_prev] > 0 else 0
    num = bigram_counts[(w_prev, w)]
    return num / denom if denom > 0 else 0.0

def mle_trigram(w_prev2, w_prev1, w):
    denom = bigram_counts[(w_prev2, w_prev1)] if bigram_counts[(w_prev2, w_prev1)] > 0 else 0
    num = trigram_counts[(w_prev2, w_prev1, w)]
    return num / denom if denom > 0 else 0.0

# Add-one (Laplace) smoothed probabilities
def add1_bigram(w_prev, w, V=V):
    num = bigram_counts[(w_prev, w)] + 1
    denom = unigram_counts[w_prev] + V
    return num / denom

def add1_trigram(w_prev2, w_prev1, w, V=V):
    num = trigram_counts[(w_prev2, w_prev1, w)] + 1
    denom = bigram_counts[(w_prev2, w_prev1)] + V
    return num / denom

# Interpolation (combine unigram, bigram, trigram)
lambda1, lambda2, lambda3 = 0.2, 0.3, 0.5  # example weights
def interp_prob(w_prev2, w_prev1, w):
    p1 = mle_unigram(w)
    p2 = mle_bigram(w_prev1, w)
    p3 = mle_trigram(w_prev2, w_prev1, w)
    return lambda1*p1 + lambda2*p2 + lambda3*p3

In [ ]:

# 5) Text generation using smoothed probabilities (add-one) and interpolation
import bisect

def sample_from_distribution(dist):
    # dist: list of (token, prob) where probs sum to >0
    tokens, probs = zip(*dist)
    total = sum(probs)
    if total <= 0:
        return random.choice(tokens)
    probs_norm = [p/total for p in probs]
    # cumulative
    cum = []
    s = 0.0
    for p in probs_norm:
        s += p
        cum.append(s)
    r = random.random()
    idx = bisect.bisect_left(cum, r)
    return tokens[min(idx, len(tokens)-1)]

def generate_line_add1(start_words=None, max_len=20):
    if start_words is None:
        w_prev2, w_prev1 = "<s>", "<s>"
    else:
        toks = start_words.split()
        if len(toks) == 1:
            w_prev2, w_prev1 = "<s>", toks[-1]
        else:
            w_prev2, w_prev1 = toks[-2], toks[-1]
    out = [] if start_words is None else toks.copy()
    for _ in range(max_len):
        # build distribution over vocabulary for next word using add-one trigram
        dist = []
        for w in list(vocab) + ["</s>"]:
            p = add1_trigram(w_prev2, w_prev1, w)
            dist.append((w, p))
        nxt = sample_from_distribution(dist)
        if nxt == "</s>":
            break
        out.append(nxt)
        w_prev2, w_prev1 = w_prev1, nxt
    return " ".join(out)

def generate_line_interp(start_words=None, max_len=20):
    if start_words is None:
        w_prev2, w_prev1 = "<s>", "<s>"
    else:
        toks = start_words.split()
        if len(toks) == 1:
            w_prev2, w_prev1 = "<s>", toks[-1]
        else:
            w_prev2, w_prev1 = toks[-2], toks[-1]
    out = [] if start_words is None else toks.copy()
    for _ in range(max_len):
        dist = []
        for w in list(vocab) + ["</s>"]:
            p = interp_prob(w_prev2, w_prev1, w)
            dist.append((w, p))
        nxt = sample_from_distribution(dist)
        if nxt == "</s>":
            break
        out.append(nxt)
        w_prev2, w_prev1 = w_prev1, nxt
    return " ".join(out)

# Examples
print("Sample generation (add-one smoothing):")
print(generate_line_add1("shall i"))
print("\nSample generation (interpolation):")
print(generate_line_interp("shall i"))

Sample generation (add-one smoothing):
shall i favour issue ey get violet bears receives precious here ndarkening ntired aloft supposing live belongs wherever came verses forgoing prophetic

Sample generation (interpolation):
shall i say nwhat merit lived for gilded tomb nand that which flies before her face dull bearer when from highmost pitch


In [ ]:
# 6) Perplexity calculation for add-one smoothed trigram and for interpolation
def perplexity_sentences_add1(sent_list):
    log_prob_sum = 0.0
    N = 0
    for s in sent_list:
        tokens = ["<s>"] + s.split() + ["</s>"]
        N += len(tokens) - 1
        for i in range(2, len(tokens)):
            w_prev2, w_prev1, w = tokens[i-2], tokens[i-1], tokens[i]
            p = add1_trigram(w_prev2, w_prev1, w)
            log_prob_sum += math.log2(p)
    ppl = 2 ** ( - (1.0 / N) * log_prob_sum )
    return ppl

def perplexity_sentences_interp(sent_list):
    log_prob_sum = 0.0
    N = 0
    for s in sent_list:
        tokens = ["<s>"] + s.split() + ["</s>"]
        N += len(tokens) - 1
        for i in range(2, len(tokens)):
            w_prev2, w_prev1, w = tokens[i-2], tokens[i-1], tokens[i]
            p = interp_prob(w_prev2, w_prev1, w)
            # smoothing for zero probs (very small floor)
            if p <= 0:
                p = 1e-12
            log_prob_sum += math.log2(p)
    ppl = 2 ** ( - (1.0 / N) * log_prob_sum )
    return ppl

# Compute perplexities on test set
ppl_add1 = perplexity_sentences_add1(test_sents[:200])  # limit to first 200 for speed
ppl_interp = perplexity_sentences_interp(test_sents[:200])

print(f"Perplexity (Add-One trigram) on test sample: {ppl_add1:.2f}")
print(f"Perplexity (Interpolation) on test sample: {ppl_interp:.2f}")

Perplexity (Add-One trigram) on test sample: 2877.61
Perplexity (Interpolation) on test sample: 17706.88



## Results & Discussion

- The notebook reports sample generated lines from both the **Add-One (Laplace) smoothed trigram model** and the **Interpolation model**.  
- It also computes **perplexity** on a held-out portion of the sonnets (sampled test set).  
- Typical observations to include in your presentation:
  - **Perplexity** tends to decrease (improve) when interpolation weights favor higher-order models but only if data supports those higher-order n-grams.
  - **Add-One smoothing** avoids zero probabilities but can over-smooth, especially with small datasets — leading to higher perplexity than more tailored smoothing methods.
  - **Generated text** from n-gram models can be locally coherent but lacks long-range structure; it will sound "Shakespeare-like" in word choice but not truly Shakespearean in meaning or meter.

In [ ]:
# 🧩 Extension: Implementing Kneser–Ney Smoothing (Advanced Section)

from collections import defaultdict

print("Implementing Kneser–Ney smoothing for bigram model...")

# Step 1: Prepare bigram and unigram counts
bigram_total = sum(bigram_counts.values())
discount = 0.75  # typical discount value

# Step 2: Continuation probabilities
# P_cont(w_i) = number of unique predecessors of w_i / total number of unique bigrams
continuation_counts = defaultdict(int)
for (w_prev, w_curr) in bigram_counts.keys():
    continuation_counts[w_curr] += 1
total_unique_bigrams = len(bigram_counts)
P_cont = {w: continuation_counts[w] / total_unique_bigrams for w in vocab}

# Step 3: Compute lambda for each w_{i-1}
# λ(w_prev) = (D / c(w_prev)) * (# of unique bigrams starting with w_prev)
unique_followers = defaultdict(set)
for (w_prev, w_curr) in bigram_counts.keys():
    unique_followers[w_prev].add(w_curr)

lambda_weight = {}
for w_prev in vocab:
    if unigram_counts[w_prev] > 0:
        lambda_weight[w_prev] = (discount / unigram_counts[w_prev]) * len(unique_followers[w_prev])
    else:
        lambda_weight[w_prev] = 0.0

# Step 4: Define Kneser-Ney probability function
def kneser_ney_prob(w_prev, w):
    c_bigram = bigram_counts[(w_prev, w)]
    c_prev = unigram_counts[w_prev]
    term1 = max(c_bigram - discount, 0) / c_prev if c_prev > 0 else 0
    term2 = lambda_weight.get(w_prev, 0) * P_cont.get(w, 0)
    return term1 + term2

# Step 5: Generate text using Kneser–Ney smoothed model
def generate_kn_line(start_word="<s>", max_len=15):
    prev = start_word
    output = [] if start_word == "<s>" else [start_word]
    for _ in range(max_len):
        dist = [(w, kneser_ney_prob(prev, w)) for w in vocab]
        total_prob = sum(p for _, p in dist)
        if total_prob == 0:
            break
        probs = [p / total_prob for _, p in dist]
        tokens = [w for w, _ in dist]
        next_word = random.choices(tokens, weights=probs, k=1)[0]
        if next_word == "</s>":
            break
        output.append(next_word)
        prev = next_word
    return " ".join(output)

# Example: Generate Shakespeare-like line using Kneser–Ney
print("\\nExample Kneser–Ney generation:")
print(generate_kn_line("shall"))

# Step 6: Compare a few probabilities
sample_pairs = [("love", "thee"), ("i", "am"), ("summer", "day")]
print("\\nComparison of probabilities (Add-One vs Kneser–Ney):")
for a, b in sample_pairs:
    p_add1 = add1_bigram(a, b)
    p_kn = kneser_ney_prob(a, b)
    print(f"P({b}|{a}) -> Add-One: {p_add1:.6f}, Kneser–Ney: {p_kn:.6f}")

Implementing Kneser–Ney smoothing for bigram model...
\nExample Kneser–Ney generation:
shall not the mind being nto of thee all should nthen in odour thence thou nor
\nComparison of probabilities (Add-One vs Kneser–Ney):
P(thee|love) -> Add-One: 0.002206, Kneser–Ney: 0.040341
P(am|i) -> Add-One: 0.006083, Kneser–Ney: 0.072169
P(day|summer) -> Add-One: 0.000328, Kneser–Ney: 0.000361


### Discussion: Kneser–Ney Smoothing

- **Purpose:** Kneser–Ney is an advanced smoothing method that improves on Add-One by redistributing probability mass based on word continuation frequency.
- **Key difference:** Instead of just counting occurrences, it also considers how many *different contexts* a word appears in.
- **Benefit:** Produces more realistic probabilities for rare word pairs, especially in smaller corpora.
- **Result:** The generated text tends to sound smoother, with less repetition.

## Conclusion

This notebook implements end-to-end using **NLTK + custom code**. It is intentionally readable and contains markdown explanations.


## References

- *Natural Language Processing in Action* — Lane, Howard, Hapke (2019)
- Project Gutenberg — Shakespeare Sonnets (used as dataset)
- NLTK documentation
- Lecture notes and online resources on n-gram modeling and smoothing
